In [1]:
import os
import sys
import numpy as np
import h5py
root = os.path.dirname(os.path.abspath(os.curdir))
sys.path.append(root)

import torch
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from torch.utils.data import Subset

from predify.utils.training import train_pcoders, eval_pcoders

from networks_2022 import BranchedNetwork
from data.ReconstructionTrainingDataset import CleanSoundsDataset
from data.ReconstructionTrainingDataset import NoisySoundsDataset

# Specify Network to train
TODO: This should be converted to a script that accepts arguments for which network to train

In [ ]:
pnet_name = 'pnet'
_train_datafile = 'clean_reconstruction_training_set'
SoundsDataset = CleanSoundsDataset
dset_kwargs = {}

In [2]:
snr = 'neg9'
pnet_name = 'pnet_snr-9'
_train_datafile = 'hyperparameter_pooled_training_dataset_random_order_noNulls'
SoundsDataset = NoisySoundsDataset
dset_kwargs = {'snr': snr}

In [3]:
from pbranchednetwork_all import PBranchedNetwork_AllSeparateHP
PNetClass = PBranchedNetwork_AllSeparateHP

# Parameters

In [4]:
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'Device: {DEVICE}')
BATCH_SIZE = 50
NUM_WORKERS = 2
PIN_MEMORY = True
NUM_EPOCHS = 70

lr = 1E-5
engram_dir = '/mnt/smb/locker/abbott-locker/hcnn/'
checkpoints_dir = f'{engram_dir}checkpoints/'
tensorboard_dir = f'{engram_dir}tensorboard/'
train_datafile = f'{engram_dir}{_train_datafile}.hdf5'

Device: cuda:0


In [5]:
!nvidia-smi

Mon Sep  5 13:57:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.48.07    Driver Version: 515.48.07    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:21:00.0 Off |                  N/A |
| 27%   26C    P8     2W / 250W |      3MiB / 11264MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Load network and optimizer

In [6]:
net = BranchedNetwork()
net.load_state_dict(torch.load(f'{engram_dir}networks_2022_weights.pt'))

/share/issa/users/es3773/hallucnn/src/models/layers.py:78: UserWarning: Inconsistent tf pad calculation in ConvLayer.
  warnings.warn('Inconsistent tf pad calculation in ConvLayer.')
/share/issa/users/es3773/hallucnn/src/models/layers.py:173: UserWarning: Inconsistent tf pad calculation: 0, 1
  warnings.warn(f'Inconsistent tf pad calculation: {pad_left}, {pad_right}')


<All keys matched successfully>

In [7]:
pnet = PNetClass(net, build_graph=True)

In [8]:
pnet.eval()

PBranchedNetwork_AllSeparateHP(
  (backbone): BranchedNetwork(
    (speech_branch): Sequential(
      (conv1): ConvLayer(
        (block): Sequential(
          (0): Conv2d(1, 96, kernel_size=(6, 14), stride=(3, 3), padding=(2, 6))
          (1): ReLU()
        )
      )
      (rnorm1): LRNorm(
        (block): LocalResponseNorm(5, alpha=0.005, beta=0.75, k=1.0)
      )
      (pool1): PoolLayer(
        (block): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
      )
      (conv2): ConvLayer(
        (block): Sequential(
          (0): Conv2d(96, 256, kernel_size=(5, 5), stride=(2, 2), padding=(1, 2))
          (1): ReLU()
        )
      )
      (rnorm2): LRNorm(
        (block): LocalResponseNorm(5, alpha=0.005, beta=0.75, k=1.0)
      )
      (pool2): PoolLayer(
        (block): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
      )
      (conv3): ConvLayer(
        (block): Sequential(
          (0): Conv2d(256, 512, kernel_size=

In [9]:
pnet.to(DEVICE)
optimizer = torch.optim.Adam(
    [{'params':getattr(pnet,f"pcoder{x+1}").pmodule.parameters(), 'lr':lr} for x in range(pnet.number_of_pcoders)],
    weight_decay=5e-4)

# Set up train/test dataset

In [10]:
train_dataset = SoundsDataset(
    train_datafile, subset=.9, **dset_kwargs)
test_dataset = SoundsDataset(
    train_datafile, subset=.9,
    train = False, **dset_kwargs
    )

In [11]:
train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY
    )
eval_loader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY
    )

# Set up checkpoints and tensorboards

In [12]:
checkpoint_path = os.path.join(checkpoints_dir, f"{pnet_name}")
if not os.path.exists(checkpoint_path):
    os.makedirs(checkpoint_path)
checkpoint_path = os.path.join(checkpoint_path, pnet_name + '-{epoch}-{type}.pth')

# summarywriter
from torch.utils.tensorboard import SummaryWriter
tensorboard_path = os.path.join(tensorboard_dir, f"{pnet_name}")
if not os.path.exists(tensorboard_path):
    os.makedirs(tensorboard_path)
sumwriter = SummaryWriter(tensorboard_path, filename_suffix=f'')

# Train

In [ ]:
loss_function = torch.nn.MSELoss()
for epoch in range(1, NUM_EPOCHS+1):
    train_pcoders(pnet, optimizer, loss_function, epoch, train_loader, DEVICE, sumwriter)
    eval_pcoders(pnet, loss_function, epoch, eval_loader, DEVICE, sumwriter)

    # save checkpoints every 5 epochs
    if epoch % 5 == 0:
        torch.save(pnet.state_dict(), checkpoint_path.format(epoch=epoch, type='regular'))

/home/es3773/.conda/envs/hcnn/lib/python3.6/site-packages/torch/nn/functional.py:3680: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  "The default behavior for interpolate/upsample with float scale_factor changed "


Training Epoch: 1 [50/13500]	Loss: 24215.2930
Training Epoch: 1 [100/13500]	Loss: 24014.3848
Training Epoch: 1 [150/13500]	Loss: 23833.8438
Training Epoch: 1 [200/13500]	Loss: 23284.7617
Training Epoch: 1 [250/13500]	Loss: 23395.9492
Training Epoch: 1 [300/13500]	Loss: 22806.3750
Training Epoch: 1 [350/13500]	Loss: 23726.7734
Training Epoch: 1 [400/13500]	Loss: 23776.9043
Training Epoch: 1 [450/13500]	Loss: 23178.8516
Training Epoch: 1 [500/13500]	Loss: 22711.7578
Training Epoch: 1 [550/13500]	Loss: 22917.3652
Training Epoch: 1 [600/13500]	Loss: 21668.8301
Training Epoch: 1 [650/13500]	Loss: 22736.4512
Training Epoch: 1 [700/13500]	Loss: 22905.9160
Training Epoch: 1 [750/13500]	Loss: 21949.5234
Training Epoch: 1 [800/13500]	Loss: 22526.7598
Training Epoch: 1 [850/13500]	Loss: 22799.1035
Training Epoch: 1 [900/13500]	Loss: 21909.0469
Training Epoch: 1 [950/13500]	Loss: 22144.9492
Training Epoch: 1 [1000/13500]	Loss: 22784.3203
Training Epoch: 1 [1050/13500]	Loss: 21287.2109
Training Epo

Training Epoch: 1 [8650/13500]	Loss: 12791.5938
Training Epoch: 1 [8700/13500]	Loss: 12535.9443
Training Epoch: 1 [8750/13500]	Loss: 12518.3164
Training Epoch: 1 [8800/13500]	Loss: 12345.1836
Training Epoch: 1 [8850/13500]	Loss: 13377.4355
Training Epoch: 1 [8900/13500]	Loss: 12303.7373
Training Epoch: 1 [8950/13500]	Loss: 12195.7910
Training Epoch: 1 [9000/13500]	Loss: 11824.3232
Training Epoch: 1 [9050/13500]	Loss: 12457.0791
Training Epoch: 1 [9100/13500]	Loss: 12547.8242
Training Epoch: 1 [9150/13500]	Loss: 12556.1084
Training Epoch: 1 [9200/13500]	Loss: 11887.7227
Training Epoch: 1 [9250/13500]	Loss: 12426.8555
Training Epoch: 1 [9300/13500]	Loss: 12023.7363
Training Epoch: 1 [9350/13500]	Loss: 12296.0537
Training Epoch: 1 [9400/13500]	Loss: 12364.1318
Training Epoch: 1 [9450/13500]	Loss: 11804.3135
Training Epoch: 1 [9500/13500]	Loss: 12186.6250
Training Epoch: 1 [9550/13500]	Loss: 11947.7441
Training Epoch: 1 [9600/13500]	Loss: 12229.6523
Training Epoch: 1 [9650/13500]	Loss: 121

Training Epoch: 2 [3700/13500]	Loss: 9228.2910
Training Epoch: 2 [3750/13500]	Loss: 8462.7822
Training Epoch: 2 [3800/13500]	Loss: 8899.3096
Training Epoch: 2 [3850/13500]	Loss: 8733.8193
Training Epoch: 2 [3900/13500]	Loss: 8685.4316
Training Epoch: 2 [3950/13500]	Loss: 8898.7393
Training Epoch: 2 [4000/13500]	Loss: 9191.3320
Training Epoch: 2 [4050/13500]	Loss: 8601.7354
Training Epoch: 2 [4100/13500]	Loss: 8559.7217
Training Epoch: 2 [4150/13500]	Loss: 9207.0840
Training Epoch: 2 [4200/13500]	Loss: 8676.0732
Training Epoch: 2 [4250/13500]	Loss: 9226.0469
Training Epoch: 2 [4300/13500]	Loss: 9368.9697
Training Epoch: 2 [4350/13500]	Loss: 8835.9854
Training Epoch: 2 [4400/13500]	Loss: 8692.5488
Training Epoch: 2 [4450/13500]	Loss: 9082.9258
Training Epoch: 2 [4500/13500]	Loss: 8356.1055
Training Epoch: 2 [4550/13500]	Loss: 8697.3613
Training Epoch: 2 [4600/13500]	Loss: 8309.4307
Training Epoch: 2 [4650/13500]	Loss: 8964.4814
Training Epoch: 2 [4700/13500]	Loss: 8840.0762
Training Epoc

Training Epoch: 2 [12400/13500]	Loss: 7248.1934
Training Epoch: 2 [12450/13500]	Loss: 7415.1006
Training Epoch: 2 [12500/13500]	Loss: 7079.3555
Training Epoch: 2 [12550/13500]	Loss: 7090.4468
Training Epoch: 2 [12600/13500]	Loss: 7480.1450
Training Epoch: 2 [12650/13500]	Loss: 7260.3823
Training Epoch: 2 [12700/13500]	Loss: 7438.0161
Training Epoch: 2 [12750/13500]	Loss: 7073.9272
Training Epoch: 2 [12800/13500]	Loss: 7211.3330
Training Epoch: 2 [12850/13500]	Loss: 7627.7729
Training Epoch: 2 [12900/13500]	Loss: 7007.8027
Training Epoch: 2 [12950/13500]	Loss: 7089.3359
Training Epoch: 2 [13000/13500]	Loss: 7186.5088
Training Epoch: 2 [13050/13500]	Loss: 6886.9897
Training Epoch: 2 [13100/13500]	Loss: 7242.1094
Training Epoch: 2 [13150/13500]	Loss: 6813.9712
Training Epoch: 2 [13200/13500]	Loss: 7160.1191
Training Epoch: 2 [13250/13500]	Loss: 7544.5923
Training Epoch: 2 [13300/13500]	Loss: 7153.8359
Training Epoch: 2 [13350/13500]	Loss: 7065.4556
Training Epoch: 2 [13400/13500]	Loss: 70

...

Training Epoch: 9 [13350/13500]	Loss: 2162.3535
Training Epoch: 9 [13400/13500]	Loss: 2182.6692
Training Epoch: 9 [13450/13500]	Loss: 2074.1396
Training Epoch: 9 [13500/13500]	Loss: 2283.0957
Training Epoch: 9 [1499/1499]	Loss: 2208.4896
Training Epoch: 10 [50/13500]	Loss: 2196.3103
Training Epoch: 10 [100/13500]	Loss: 2141.3936
Training Epoch: 10 [150/13500]	Loss: 2079.2891
Training Epoch: 10 [200/13500]	Loss: 2139.1951
Training Epoch: 10 [250/13500]	Loss: 2163.6016
Training Epoch: 10 [300/13500]	Loss: 2080.6599
Training Epoch: 10 [350/13500]	Loss: 2265.6917
Training Epoch: 10 [400/13500]	Loss: 2303.3562
Training Epoch: 10 [450/13500]	Loss: 2159.7222
Training Epoch: 10 [500/13500]	Loss: 2091.5217
Training Epoch: 10 [550/13500]	Loss: 2234.4626
Training Epoch: 10 [600/13500]	Loss: 2037.8315
Training Epoch: 10 [650/13500]	Loss: 2167.1069
Training Epoch: 10 [700/13500]	Loss: 2202.7495
Training Epoch: 10 [750/13500]	Loss: 2106.9614
Training Epoch: 10 [800/13500]	Loss: 2112.7947
Training Ep

Training Epoch: 10 [8400/13500]	Loss: 2157.1609
Training Epoch: 10 [8450/13500]	Loss: 2036.1162
Training Epoch: 10 [8500/13500]	Loss: 2074.8569
Training Epoch: 10 [8550/13500]	Loss: 2018.3368
Training Epoch: 10 [8600/13500]	Loss: 2011.2611
Training Epoch: 10 [8650/13500]	Loss: 2051.5083
Training Epoch: 10 [8700/13500]	Loss: 1978.0128
Training Epoch: 10 [8750/13500]	Loss: 2000.6646
Training Epoch: 10 [8800/13500]	Loss: 1961.9133
Training Epoch: 10 [8850/13500]	Loss: 2244.6072
Training Epoch: 10 [8900/13500]	Loss: 1929.0596
Training Epoch: 10 [8950/13500]	Loss: 1940.0547
Training Epoch: 10 [9000/13500]	Loss: 1840.6223
Training Epoch: 10 [9050/13500]	Loss: 1998.2638
Training Epoch: 10 [9100/13500]	Loss: 2041.0179
Training Epoch: 10 [9150/13500]	Loss: 2045.8025
Training Epoch: 10 [9200/13500]	Loss: 1948.7565
Training Epoch: 10 [9250/13500]	Loss: 2029.3911
Training Epoch: 10 [9300/13500]	Loss: 1927.6165
Training Epoch: 10 [9350/13500]	Loss: 2023.6263
Training Epoch: 10 [9400/13500]	Loss: 20

Training Epoch: 11 [3350/13500]	Loss: 1861.6595
Training Epoch: 11 [3400/13500]	Loss: 1868.6370
Training Epoch: 11 [3450/13500]	Loss: 1839.1016
Training Epoch: 11 [3500/13500]	Loss: 1864.1066
Training Epoch: 11 [3550/13500]	Loss: 1806.6506
Training Epoch: 11 [3600/13500]	Loss: 1871.4432
Training Epoch: 11 [3650/13500]	Loss: 1897.9745
Training Epoch: 11 [3700/13500]	Loss: 1939.6031
Training Epoch: 11 [3750/13500]	Loss: 1779.6038
Training Epoch: 11 [3800/13500]	Loss: 1866.0944
Training Epoch: 11 [3850/13500]	Loss: 1842.8286
Training Epoch: 11 [3900/13500]	Loss: 1834.4620
Training Epoch: 11 [3950/13500]	Loss: 1866.6251
Training Epoch: 11 [4000/13500]	Loss: 1923.7565
Training Epoch: 11 [4050/13500]	Loss: 1804.8154
Training Epoch: 11 [4100/13500]	Loss: 1786.3018
Training Epoch: 11 [4150/13500]	Loss: 1946.4731
Training Epoch: 11 [4200/13500]	Loss: 1845.3743
Training Epoch: 11 [4250/13500]	Loss: 1931.7200
Training Epoch: 11 [4300/13500]	Loss: 1977.6515
Training Epoch: 11 [4350/13500]	Loss: 18

Training Epoch: 11 [11850/13500]	Loss: 1687.1229
Training Epoch: 11 [11900/13500]	Loss: 1705.8483
Training Epoch: 11 [11950/13500]	Loss: 1740.6226
Training Epoch: 11 [12000/13500]	Loss: 1782.2686
Training Epoch: 11 [12050/13500]	Loss: 1707.6802
Training Epoch: 11 [12100/13500]	Loss: 1738.1461
Training Epoch: 11 [12150/13500]	Loss: 1673.9547
Training Epoch: 11 [12200/13500]	Loss: 1686.3401
Training Epoch: 11 [12250/13500]	Loss: 1660.8983
Training Epoch: 11 [12300/13500]	Loss: 1685.7106
Training Epoch: 11 [12350/13500]	Loss: 1728.6477
Training Epoch: 11 [12400/13500]	Loss: 1734.9696
Training Epoch: 11 [12450/13500]	Loss: 1741.5621
Training Epoch: 11 [12500/13500]	Loss: 1689.4727
Training Epoch: 11 [12550/13500]	Loss: 1692.6260
Training Epoch: 11 [12600/13500]	Loss: 1764.0852
Training Epoch: 11 [12650/13500]	Loss: 1749.5338
Training Epoch: 11 [12700/13500]	Loss: 1747.6957
Training Epoch: 11 [12750/13500]	Loss: 1697.1925
Training Epoch: 11 [12800/13500]	Loss: 1727.8772
Training Epoch: 11 [

Training Epoch: 12 [6850/13500]	Loss: 1568.5223
Training Epoch: 12 [6900/13500]	Loss: 1577.0581
Training Epoch: 12 [6950/13500]	Loss: 1592.9443
Training Epoch: 12 [7000/13500]	Loss: 1724.5928
Training Epoch: 12 [7050/13500]	Loss: 1716.8699
Training Epoch: 12 [7100/13500]	Loss: 1641.9529
Training Epoch: 12 [7150/13500]	Loss: 1667.6716
Training Epoch: 12 [7200/13500]	Loss: 1542.3323
Training Epoch: 12 [7250/13500]	Loss: 1628.7114
Training Epoch: 12 [7300/13500]	Loss: 1611.3744
Training Epoch: 12 [7350/13500]	Loss: 1725.7098
Training Epoch: 12 [7400/13500]	Loss: 1576.6295
Training Epoch: 12 [7450/13500]	Loss: 1590.0934
Training Epoch: 12 [7500/13500]	Loss: 1603.7709
Training Epoch: 12 [7550/13500]	Loss: 1654.8380
Training Epoch: 12 [7600/13500]	Loss: 1652.3837
Training Epoch: 12 [7650/13500]	Loss: 1677.1257
Training Epoch: 12 [7700/13500]	Loss: 1613.9709
Training Epoch: 12 [7750/13500]	Loss: 1670.1625
Training Epoch: 12 [7800/13500]	Loss: 1641.3079
Training Epoch: 12 [7850/13500]	Loss: 16